## New York City Crime Statistics - Web-Scrapping Project

This code has been written in **Python 3.8.13** and executed on a M1 Mac \
Depending on the OS used for execution a different Chrome Driver might be needed to execute succesfully

<img src="Images/Compstat 2.0 Example.png"></img>

In the image above, we can see the Compstat 2.0 Website. On the left of the website, in the **CompStat Book** 
section, it can be specified what data points should be shown on the **Incident Map** in the middle. (In the 
image above, I selected the **<u>Total</u>** Number of Crimes that Happened in the **<u>Week to Date</u>** 
**<u>2022</u>** from 06/13/2022 - 06/19/2022). With each circle, there are relational x and y coordinates 
as well as a specific and broad cime label and the time of the occurence of the crime. Each circle on the 
map has to be clicked individually to derive the previously mentioned data from the HTML code of the website.

### Objective
- first, specify a category and time frame of crimes that should be displayed on the map
- loop through all circles on the map to get the time, location, and specific/broad crime label

### Implementation

In [ ]:
!pip3 install selenium webdriver-manager tqdm pandas geopandas datetime keplergl

- importing all relevant packages for the scrapping of the Compstat 2.0 website

In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import numpy as np
import time as tm
import re
from datetime import datetime

from tqdm import tqdm

- The following function sets up a chrome instance to use for the website interactions

In [2]:
def settingUpChrome():
    """Creates an instance of the chrome browser and opens up the Compstat2.0 website"""
    #option = webdriver.ChromeOptions()
    #option.add_argument('headless')
    global driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://compstat.nypdonline.org/")
    driver.maximize_window()

- This function finds the relevant elements on the Compstat 2.0 website and assigns them to global variables so that they can be interacted with.
- It also creates an Dictionary where all elements that specify the different types of data categories are being saved.

In [19]:
def loadElementsOnSite():
    """Loads all neccesary elements on the site that will be interacted with and assigns them to variables"""
    global wait
    wait = WebDriverWait(driver, 20)
    global wait_for_element
    def wait_for_element(locator_type:str,locator:str):
            return wait.until(EC.presence_of_element_located((eval(locator_type), locator))) 

    global search_category_dict 
    search_category_dict = {}

    global general_element_dictionary
    general_element_dictionary = {}

    try:
        # Totals
        global total_week_to_date_2022
        total_week_to_date_2022 = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[9]/td[1]/div/div')

        search_category_dict["total_week_to_date_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[9]/td[2]/div/div')

        search_category_dict["total_28_day_2022"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[9]/td[4]/div/div')

        search_category_dict["total_28_day_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[9]/td[5]/div/div')

        global total_year_to_date_2022  
        total_year_to_date_2022 = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[9]/td[7]/div/div')

        search_category_dict["total_year_to_date_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[9]/td[8]/div/div')

        # Patrols
        search_category_dict["patrol_week_to_date_2022"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[10]/td[1]/div/div')

        search_category_dict["patrol_week_to_date_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[10]/td[2]/div/div')

        search_category_dict["patrol_28_day_2022"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[10]/td[4]/div/div')

        search_category_dict["patrol_28_day_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[10]/td[5]/div/div')

        search_category_dict["patrol_year_to_date_2022"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[10]/td[7]/div/div')
        
        search_category_dict["patrol_year_to_date_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[10]/td[8]/div/div')

        # Transits
        search_category_dict["transit_week_to_date_2022"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[11]/td[1]/div/div')

        search_category_dict["transit_week_to_date_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[11]/td[2]/div/div')

        search_category_dict["transit_28_day_2022"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[11]/td[4]/div/div')

        search_category_dict["transit_28_day_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[11]/td[5]/div/div')

        search_category_dict["transit_year_to_date_2022"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[11]/td[7]/div/div')
        
        search_category_dict["transit_year_to_date_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[11]/td[8]/div/div')

        # Housings
        search_category_dict["housing_week_to_date_2022"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[12]/td[1]/div/div')

        search_category_dict["housing_week_to_date_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[12]/td[2]/div/div')

        search_category_dict["housing_28_day_2022"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[12]/td[4]/div/div')

        search_category_dict["housing_28_day_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[12]/td[5]/div/div')

        search_category_dict["housing_year_to_date_2022"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[12]/td[7]/div/div')
        
        search_category_dict["housing_year_to_date_2021"] = wait_for_element("By.XPATH",\
             '//*[@id="report-win1"]/div[3]/div[2]/div/div/div[3]/div[1]/table/tbody/tr[12]/td[8]/div/div')


        global wait_for_cicles_to_appear
        wait_for_cicles_to_appear = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR,\
                 '#report-win13 > div.window-content > div.full-height > div > div.full-height.k-map > div.km-widget.km-scroll-wrapper > div.km-scroll-container > div:nth-child(4) > svg > circle:nth-child(2)')))

        global first_circle
        first_circle = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR,\
                 '#report-win13 > div.window-content > div.full-height > div > div.full-height.k-map > div.km-widget.km-scroll-wrapper > div.km-scroll-container > div:nth-child(4) > svg')))                                                     

        global zoom_out
        zoom_out = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR,\
                 '#report-win13 > div.window-content > div.full-height > div > div.full-height.k-map > div.k-map-controls.k-pos-top.k-pos-left > div > button.k-button.k-zoom-out')))

        global zoom_in
        zoom_in = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR,\
                 '#report-win13 > div.window-content > div.full-height > div > div.full-height.k-map > div.k-map-controls.k-pos-top.k-pos-left > div > button.k-button.k-zoom-in')))

        global incident_map_logo
        incident_map_logo = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR,\
                 '#report-win13 > div.window-header > h4')))
        
        global map_mover
        map_mover = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR,\
                 '#report-win13 > div.window-content > div.full-height > div > div.full-height.k-map > div.km-widget.km-scroll-wrapper > div.km-scroll-container'))) 
        
        global toggle_large_map
        toggle_large_map = wait.until(
            EC.presence_of_element_located((By.XPATH,\
                 '//*[@id="report-win13"]/div[2]/a/span')))

        global map_element
        map_element = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR,\
                 '#report-win13 > div.window-content > div.full-height > div > div.full-height.k-map')))

    finally:
        print("loading of Elements has finished")

This function allows to move on the SVG map horizontaly and vertically

In [4]:
def moveOnMap(direction: str, scalar):
    """allows horizontal and vertical movement on the map; magnitude of movement is controlled by the scalar"""
    map_element = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#report-win13 > div.window-content > div.full-height > div > div.full-height.k-map')))

    if direction == "moveUp":
        webdriver.ActionChains(driver).move_to_element(map_element).click_and_hold().move_by_offset(0,1*scalar).release().perform() 
    elif direction == "moveDown":
        webdriver.ActionChains(driver).move_to_element(map_element).click_and_hold().move_by_offset(0,-1*scalar).release().perform()
    elif direction == "moveLeft":    
        webdriver.ActionChains(driver).move_to_element(map_element).click_and_hold().move_by_offset(1*scalar,0).release().perform() 
    elif direction == "moveRight":    
        webdriver.ActionChains(driver).move_to_element(map_element).click_and_hold().move_by_offset(-1*scalar,0).release().perform()
    else:
        print("please specify a direction that can be interpreted")    

This function does a number of things:
- enlarges the SVG map to increase the viewing field of circles on the map
- on the map zooms out a bit to make sure all circles are within the visible frame of the map (hence clickable)
- uses JavaScript to set the `.style.visibility` of every circle to `hidden`
- then, we iterate through the circles, and set their visibility to `visible` one after the other and click the circle on the map
    - 1. We do it this way because the data we are looking for only appears in the HTML for a single circle (the one that has been clicked)
    - 2. We make every circle invisible so that there is no other circle that overlaps the circle that we intend to click
- after a circle has been clicked and the data has been extracted, we set the visibility back to `hidden` and move to the next circle


In [29]:
def selectDataOfChategory(verbose: bool):
    """
    Parameters:
    category (WebElement): can be one of the cells on the left side of the Compstat2.0 website that specify agregate numbers of crimes

    Returns:
    DataFrame: of all crimes that belong to the given category that has been specified before including: (location, time, specific crime label, broad crime label)
    """
    def map_loading_disapear():
        loading_disapear = wait.until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR,\
                '#report-win13 > div.k-loading-mask > div.k-loading-image')))


    def make_all_disapear():
        circles = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR,\
                    '#report-win13 > div.window-content > div.full-height > div > div.full-height.k-map > div.km-widget.km-scroll-wrapper > div.km-scroll-container > div:nth-child(4) > svg > circle')))
        
        driver.execute_script("var all_circles = document.getElementsByTagName('circle');\
                            for (i = 0; i < all_circles.length; i++){all_circles[i].style.visibility = 'hidden'};")

        

    try:
        ########
        
        total_year_to_date_2022.click()

        ########


        map_loading_disapear()
        toggle_large_map.click()
        tm.sleep(0.2)
        zoom_out.click()
        moveOnMap("moveRight", 72)
        webdriver.ActionChains(driver).move_to_element(first_circle).click(first_circle).perform()
        #moveOnMap("moveRight", 70)

        circles = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#report-win13 > div.window-content > div.full-height > div > div.full-height.k-map > div.km-widget.km-scroll-wrapper > div.km-scroll-container > div:nth-child(4) > svg > circle'))
        )                                                          

        print("number of circles: ", len(circles)) if verbose else None

        global data_frame
        data_frame = pd.DataFrame(columns=["x_cord","y_cord","time","crime","crime_category"])
        loop_crime_counter = 0
        text_crime_counter = 0

        make_all_disapear()
        
        for i in tqdm(range(len(circles))):

            

            print("circle number: ", i) if verbose else None
            
            driver.execute_script(f"var all_circles = document.getElementsByTagName('circle');\
                                    all_circles[{i}].style.visibility = 'visible';")

            given_circle = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, f'#report-win13 > div.window-content > div.full-height > div > div.full-height.k-map > div.km-widget.km-scroll-wrapper > div.km-scroll-container > div:nth-child(4) > svg > circle:nth-child({i+2})'))
            )

            webdriver.ActionChains(driver).move_to_element(given_circle).click(given_circle).perform()

            y_coordinate = given_circle.get_attribute("cy")
            x_coordinate = given_circle.get_attribute("cx")

            information_box = wait.until(
                EC.presence_of_element_located((By.ID, "map-popup-13")))

            print() if verbose else None
            print() if verbose else None
            print("------COORDINATES------") if verbose else None
            print(i, given_circle.get_attribute("outerHTML")) if verbose else None
            print() if verbose else None
            print("y_coordinate: ", y_coordinate) if verbose else None
            print("x_coordinate: ", x_coordinate) if verbose else None

            try:
                #if there is a h6 header in the informationbox then there are multiple crimes at the same location
                h6_header = information_box.find_element(By.TAG_NAME, "h6")

            except:
                h6_header = False

            if h6_header:
                n_of_crimes = int(re.findall(r'-?\d+\.?\d*', h6_header.get_attribute("innerHTML"))[0])
                text_crime_counter += n_of_crimes

                multi_crime_element = information_box.find_elements(By.XPATH, '//div[@style="border-bottom: 1px solid gray;"]')
                
                for j, element in enumerate(multi_crime_element):
                    loop_crime_counter += 1
                    print(element.get_attribute("innerHTML")) if verbose else None
                    t = 0
                    tm.sleep(0.2)
                    
                    crime_label_HTML = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,f'#map-popup-13 > div > div:nth-child({j+2}) > div:nth-child({t+1}) > span')))
                    
                    
                    if len(crime_label_HTML.get_attribute("title"))>0:
                        crime_label = crime_label_HTML.get_attribute("title") 
                    else:
                        crime_label = crime_label_HTML.text
            
                    print(crime_label) if verbose else None

                    crime_category_label_HTML = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#map-popup-13 > div > div:nth-child(1) > h5')))
                    crime_category_label = crime_category_label_HTML.text
                    
                    date_time_label_HTML = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,f"#map-popup-13 > div > div:nth-child({j+2}) > div:nth-child({t+2}) > span")))
                    date_time_label = date_time_label_HTML.text

                    data_frame.loc[loop_crime_counter] = pd.Series({"x_cord":x_coordinate,"y_cord":y_coordinate,"time":date_time_label,"crime":crime_label, "crime_category":crime_category_label})
                    print(loop_crime_counter) if verbose else None

            else:
                print("single crime: ----> ") if verbose else None
                loop_crime_counter += 1
                text_crime_counter += 1

                tm.sleep(0.2)
                crime_label_HTML = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'#map-popup-13 > div > div:nth-child(2) > div:nth-child(1) > span')))

                print("thats what I'm looking for: ", crime_label_HTML.text) if verbose else None


                if len(crime_label_HTML.get_attribute("title"))>0:
                    crime_label = crime_label_HTML.get_attribute("title") 
                else:
                    crime_label = crime_label_HTML.text

                crime_category_label_HTML = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#map-popup-13 > div > div:nth-child(1) > h5')))
                crime_category_label = crime_category_label_HTML.text

                print(crime_label) if verbose else None

                date_time_label_HTML = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'#map-popup-13 > div > div:nth-child(2) > div:nth-child(2) > span')))
                date_time_label = date_time_label_HTML.text

                data_frame.loc[loop_crime_counter] = pd.Series({"x_cord":x_coordinate,"y_cord":y_coordinate,"time":date_time_label,"crime":crime_label, "crime_category":crime_category_label})


                print("loop_crime_counter: ", loop_crime_counter) if verbose else None
                print("text_crime_counter: ", text_crime_counter) if verbose else None
                
            print() if verbose else None
            print() if verbose else None
            print("------ATTRIBUTES------") if verbose else None
            print(information_box.get_attribute("innerHTML")) if verbose else None
            print() if verbose else None
            print() if verbose else None

            driver.execute_script(f"var all_circles = document.getElementsByTagName('circle');\
                                    all_circles[{i}].style.visibility = 'hidden';")
            
            incident_map_logo.click()

    finally:
        print("complete")
        driver.quit()

executing all three function together

In [37]:
settingUpChrome()
loadElementsOnSite()
selectDataOfChategory(verbose=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64_m1.zip
[WDM] - Driver has been saved in cache [/Users/suntim/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53]


loading of Elements has finished


 87%|████████▋ | 22848/26170 [12:35:07<1:49:47,  1.98s/it]


complete


WebDriverException: Message: disconnected: unable to send message to renderer
  (Session info: chrome=103.0.5060.53)
Stacktrace:
0   chromedriver                        0x0000000102c11d14 chromedriver + 3792148
1   chromedriver                        0x0000000102ba8828 chromedriver + 3360808
2   chromedriver                        0x00000001028a90b8 chromedriver + 217272
3   chromedriver                        0x0000000102895e3c chromedriver + 138812
4   chromedriver                        0x00000001028963d0 chromedriver + 140240
5   chromedriver                        0x00000001028ad6f8 chromedriver + 235256
6   chromedriver                        0x0000000102909e10 chromedriver + 613904
7   chromedriver                        0x0000000102901a60 chromedriver + 580192
8   chromedriver                        0x00000001028ce318 chromedriver + 369432
9   chromedriver                        0x0000000102be71e8 chromedriver + 3617256
10  chromedriver                        0x0000000102beb678

In [34]:
from IPython.core.display import display 
 
pd.set_option("display.max_rows", None)


def display_():    
    pd.set_option("display.max_rows", None)

/var/folders/t5/897mfw654t12zzv623vx_9r80000gn/T/ipykernel_19711/1547930017.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [ ]:
display(data_frame)

In [39]:
data_frame.head()

,x_cord,y_cord,time,crime,crime_category
1,832.2644612831646,403.5159037694975,06/25/22 6PM,"1,UNCLASSIFIED",Robbery
2,832.2644612831646,403.5159037694975,06/25/22 12PM,"COMMERCIAL,DAY",Robbery
3,832.2644612831646,403.5159037694975,06/24/22 7AM,BEGIN AS SHOPLIFTING,Robbery
4,832.2644612831646,403.5159037694975,06/23/22 5PM,"GRAND FROM RETAIL STORE, UNATTENDED",Robbery
5,832.2644612831646,403.5159037694975,06/21/22 6PM,GRAND FROM STORE-SHOPL,Robbery


In [43]:
data_frame.to_csv('scraped-data/incomplete_total_year_to_date_2022.csv')
data_frame.to_excel("scraped-data/incomplete_total_year_to_date_2022.xlsx")

In [49]:
data_frame.head()

,x_cord,y_cord,time,crime,crime_category
1,832.2644612831646,403.5159037694975,06/25/22 6PM,"1,UNCLASSIFIED",Robbery
2,832.2644612831646,403.5159037694975,06/25/22 12PM,"COMMERCIAL,DAY",Robbery
3,832.2644612831646,403.5159037694975,06/24/22 7AM,BEGIN AS SHOPLIFTING,Robbery
4,832.2644612831646,403.5159037694975,06/23/22 5PM,"GRAND FROM RETAIL STORE, UNATTENDED",Robbery
5,832.2644612831646,403.5159037694975,06/21/22 6PM,GRAND FROM STORE-SHOPL,Robbery


In [2]:
data_frame = pd.read_csv("scraped-data/incomplete_total_year_to_date_2022.csv")

In [48]:
data_frame.x_cord.unique()

array([ 832.26446128,  841.87764585,  841.70288319, ...,  871.28408596,
       1000.37209279,  859.42134178])

In [59]:
print(data_frame.y_cord.unique())

[403.51590377 467.87840414 396.86178166 ... 491.39075174 491.55806266
 490.97875994]


In [3]:
test_data_frame = data_frame

In [22]:
test_data_frame.columns

Index(['Unnamed: 0', 'Longitude', 'Latitude', 'time', 'crime',
       'crime_category'],
      dtype='object')

In [4]:
test_data_frame = test_data_frame.rename(columns={"x_cord":"Longitude","y_cord":"Latitude"})

In [5]:
test_data_frame["Latitude"] = test_data_frame["Latitude"].apply(lambda y: -0.001040301*float(y)+41.17080609)
test_data_frame["Longitude"] = test_data_frame["Longitude"].apply(lambda x: 0.001373969*float(x)-75.13412029)



In [23]:
test_data_frame["time"] = test_data_frame["time"].apply(lambda time: datetime.strptime(time, '%m/%d/%y %I%p'))

In [24]:
test_data_frame.head(3)

,Unnamed: 0,Longitude,Latitude,time,crime,crime_category
0,1,-73.990615,40.751028,2022-06-25 18:00:00,"1,UNCLASSIFIED",Robbery
1,2,-73.990615,40.751028,2022-06-25 12:00:00,"COMMERCIAL,DAY",Robbery
2,3,-73.990615,40.751028,2022-06-24 07:00:00,BEGIN AS SHOPLIFTING,Robbery


In [26]:
map_2.add_data(data=test_data_frame,name="test_data_frame")

In [28]:
test_data_frame.to_json("scraped-data/long_lat_incomplete_total_year_to_date_2022")
test_data_frame.to_csv("scraped-data/long_lat_incomplete_total_year_to_date_2022.csv")

In [118]:
!which pip

/Users/suntim/opt/miniconda3/envs/python3.10.4/bin/pip


In [6]:
custom_config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [],
      "interactionConfig": {}
    },
    "mapState": {
      "bearing": -4.928571428571431,
      "dragRotate": True,
      "latitude": 40.75101599999,
      "longitude": -73.9352420000,
      #"pitch": 49.18440507924836,
      "zoom": 12,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": True,
        "building": True,
        "water": True,
        "land": True
      }
    }
  }
}

In [20]:
date_time_str = '02/14/22 10PM'

date_time_obj = datetime.strptime(date_time_str, '%m/%d/%y %I%p')


print ("The type of the date is now",  type(date_time_obj))
print ("The date is", date_time_obj)

The type of the date is now <class 'datetime.datetime'>
The date is 2022-02-14 22:00:00


In [25]:
from keplergl import KeplerGl
map_2 = KeplerGl(height=800,config=custom_config)
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [], 'interactionConfig': {}…

In [52]:
test_data_frame.Longitude.unique()

array([-73.99062892, -73.97742229, -73.97766238, ..., -73.93702364,
       -73.75968215, -73.95332071])

In [65]:
test_data_frame.head()

,Unnamed: 0,Longitude,Latitude,time,crime,crime_category
0,1,-75.235781,41.128413,06/25/22 6PM,"1,UNCLASSIFIED",Robbery
1,2,-75.235781,41.128413,06/25/22 12PM,"COMMERCIAL,DAY",Robbery
2,3,-75.235781,41.128413,06/24/22 7AM,BEGIN AS SHOPLIFTING,Robbery
3,4,-75.235781,41.128413,06/23/22 5PM,"GRAND FROM RETAIL STORE, UNATTENDED",Robbery
4,5,-75.235781,41.128413,06/21/22 6PM,GRAND FROM STORE-SHOPL,Robbery


In [10]:
total_week_to_date_2022 = pd.read_csv("scraped-data/total_week_to_date_2022.csv")

- converting the SVG coordinates to longitude and latitudes

In [ ]:
total_week_to_date_2022.head(20)

In [75]:
data_frame[data_frame["Unnamed: 0"]==33471]

,Unnamed: 0,x_cord,y_cord,time,crime,crime_category
33470,33471,1023.574167,464.402568,05/02/22 5PM,GRAND BY BANK ACCT COMPROMISE-UNCLASSIFIED,Gr. Larceny


In [72]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Marker

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(40.760610, -73.935242),
    zoom=12
    )

def pointOnMap(SVGtoLL):
    coords = SVGtoLL

    longi_num =  SVGtoLL[0][0]
    longi_denum = SVGtoLL[0][1]

    lati_num = SVGtoLL[1][0]
    lati_denum = SVGtoLL[1][1]

    global m
    m.add_layer(Marker(location = (40.751016,-73.935242)))


#for i in range(len(test_data_frame)):
#    m.add_layer(Marker(location = (test_data_frame["y_cord"][i], test_data_frame["x_cord"][i]), name = "LL", color = "red"))
#pointOnMap(SVGtoLL(140.1489072337863,358.6947293562116))
m


Map(center=[40.76061, -73.935242], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

### In Planning

- database implementation
- parallel execution via multiple taps to increase scrapping speed
- data exploration
- data visualization
- machine-learning application